In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U \
langchain-community==0.3.7 \
pymupdf==1.24.14 \
faiss-gpu==1.7.2 \
SQLAlchemy==2.0.35 \
dataclasses-json==0.6.7 \
httpx-sse==0.4.0 \
pydantic-settings==2.6.1 \
marshmallow==3.23.1 \
python-dotenv==1.0.1 \
typing-inspect==0.9.0 \
mypy-extensions==1.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.2.10
    Uninstalling langsmith-0.2.10:
      Successfully uninstalled langsmith-0.2.10


In [ ]:
import torch
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import logging
from langchain_core.prompts import PromptTemplate
from typing import List
from langchain_core.output_parsers import BaseOutputParser

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('langchain.retrievers.multi_query')
logger.setLevel(logging.INFO)

# PDF 로드 및 텍스트 분할
loader = PyMuPDFLoader("/content/drive/MyDrive/Colab Notebooks/pdf/BBS_202402151054353090.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
text_contents = [doc.page_content for doc in texts]

# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 벡터 스토어 설정
vectorstore = FAISS.from_texts(text_contents, embeddings)

# LLM 설정 (Qwen/Qwen2.5-1.5B-Instruct 사용)
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
llm = HuggingFacePipeline(
    pipeline=pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        device=0 if torch.cuda.is_available() else -1
    )
)

# 사용자 정의 프롬프트 템플릿 생성
custom_prompt = PromptTemplate(
    input_variables=["question"],
    template="""당신은 AI 언어 모델 어시스턴트입니다. 사용자가 제공한 질문에 대해 벡터 데이터베이스에서 관련 문서를 검색할 수 있도록 질문을 3가지 다른 버전으로 생성하는 것이 당신의 임무입니다.
사용자의 질문을 다양한 관점에서 재구성하여 거리 기반 유사도 검색의 한계를 극복할 수 있도록 돕는 것이 목표입니다.
각 버전의 질문은 줄바꿈으로 구분하여 작성하세요.
한국어로 작성하세요. 원본 질문: {question}"""
)

# 출력 파서 정의
class LineListOutputParser(BaseOutputParser):
    def parse(self, text: str) -> List[str]:
        return text.strip().split("\n")

# LLM 체인 생성
output_parser = LineListOutputParser()
llm_chain = custom_prompt | llm | output_parser

# 다중 쿼리 리트리버 설정
retriever_from_llm = MultiQueryRetriever(
    retriever=vectorstore.as_retriever(),
    llm_chain=llm_chain,
    parser_key="lines"
)
retriever_from_llm.verbose = True

# 쿼리 실행
query = "국내 고령화 전망에 대해 알려주세요"
results = retriever_from_llm.get_relevant_documents(query)

# 결과 출력
for i, doc in enumerate(results[:5]):  # 상위 5개 결과만 출력
    print(f"관련 문서 {i+1}:")
    print(doc.page_content[:100] + "...\n")

<ipython-input-4-07236f1be989>:27: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-4-07236f1be989>:36: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(
<ipython-input-4-07236f1be989>:77: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever_from_llm.get_relevant_documents(query)
INFO:langchain.retrievers.multi_query:Generated queries: ['당신은 AI 언어 모델 어시스턴트입니다. 사용자가 제공한 질문에 대해 벡터 데이터베이스에서 관련 문서를 검색할 수 있도록 질문을 3가지 다른 버전으로 생성하는 것이 당신의 임무입니다.', '사용자의 질문을 다양한 관점에서 재구성하여 거리 기반 유사도 검색의 한계를 극복할 수 있도록 돕는 것이 목표입니다.', '각 버전의 질문은 줄바꿈으로 구분하여 작성하세요.',

관련 문서 1:
TIPA 이슈 리포트
Vol. 6
- 14 -
Ⅲ. 고령화 기술·제품 동향
◎ 신체활동/이동 보조 분야
 (한국, 스마트 지팡기) 말하는 스마트 지팡기 ‘톡톡스틱’은 기존 지팡이...

관련 문서 2:
 (일본, DMM PALMI) 노인용 대화로봇으로써 분위기 등을 감지하여 상대의 말 예측 
가능하고, 데이터가 쌓이면 말투를 비롯한 말솜씨가 발전, 상대방의 과거 대화내용·취미 ...

관련 문서 3:
고령화시대 해결을 위한 기술개발
- 15 -
[ 보행 재활기구, 트리 ]
[ 배설지원 로봇 ]
* 출처: : Shinkachi-portal
* 출처: 일본 사이버다인, 배설지원 로...

관련 문서 4:
정기적으로 전달하고 응답내용을 분석하여 위험에 처하거나 특별한 주의가 필요하다고 인식되면 응급 
알람 메시지를 가족이나 의사에게 전달
▪ 시니어의 건강상태가 좋지않아 즉각적인 도움...

관련 문서 5:
▪ 돌봄 분야 그동안의 개발은 공급자 중심으로 이루어졌지만, 수많은 기술개발의 성공에도 사업화로 
이어진 사례는 미미하였으며, 주요 원인으로 수요자의 요구와 피드백을 직접적이고 상...

